In [1]:

import pandas as pd
import numpy as np
from vmdpy import VMD
import torch
from joblib import dump, load
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'


train_xdata = load('train_xdata')
val_xdata = load('val_xdata')
test_xdata = load('test_xdata')
train_ylabel = load('train_ylabel')
val_ylabel = load('val_ylabel')
test_ylabel = load('test_ylabel')

# -----VMD para--------
alpha = 2000
tau = 0 
DC = 0 
init = 1 
tol = 1e-7
# -----para--------

# VMD + FFT 数据集的制作
def make_feature_datasets(data, imfs_k):

    samples = data.shape[0]
    signl_len = data.shape[1]

    data = np.array(data)


    features_num = imfs_k + 1

    features = np.zeros((samples, features_num, signl_len))  


    for i in range(samples):

        # Apply VMD
        u, u_hat, omega= VMD(data[i], alpha, tau, imfs_k, DC, init, tol)


        fft_result1 = np.fft.fft(data[i])
        magnitude_spectrum = np.abs(fft_result1)
        magnitude_spectrum = magnitude_spectrum.reshape(1, -1)


        combined_matrix = np.vstack((u, magnitude_spectrum))
        features[i] = combined_matrix


    features = torch.tensor(features).float()
    return features


K = 4

train_features = make_feature_datasets(train_xdata, K)
val_features = make_feature_datasets(val_xdata, K)
test_features = make_feature_datasets(test_xdata, K)


dump(train_features, 'train_features_vmd_fft')
dump(val_features, 'val_features_vmd_fft')
dump(test_features, 'test_features_vmd_fft')

['test_features_vmd_fft']

In [2]:
print('data shape:')
print(train_features.shape, train_ylabel.shape)
print(val_features.shape, val_ylabel.shape)
print(test_features.shape, test_ylabel.shape)

data shape:
torch.Size([7161, 5, 1024]) torch.Size([7161])
torch.Size([2046, 5, 1024]) torch.Size([2046])
torch.Size([1023, 5, 1024]) torch.Size([1023])
